# Vertex AI - Feature Store

**Prerequisites**
- `00 - Initial Setup`
- `01 - BigQuery - Data`
- `06 - Vertex AI - Training Pipeline and Serving`
    - for online prediction this used the endpoing defined in 06
    
**Resources**
- Based on:
    - https://cloud.google.com/vertex-ai/docs/featurestore/managing-featurestores
- API Documentation:
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/services.html
    - https://googleapis.dev/python/aiplatform/latest/aiplatform_v1beta1/featurestore_service.html

**Overview**

<img src="architectures/statmike-mlops-07.png">


---
## Setup

Update Python library for aiplatform (Vertex AI)
- Restart the kernel: Menus > Kernel > Restart Kernel

In [2]:
!pip install --upgrade git+https://github.com/googleapis/python-aiplatform.git@main-test -q

Import Libraries

In [1]:
from google.cloud.aiplatform_v1beta1 import (FeaturestoreOnlineServingServiceClient, FeaturestoreServiceClient, types)
from google.protobuf.duration_pb2 import Duration
from google.protobuf.timestamp_pb2 import Timestamp
from google.protobuf.field_mask_pb2 import FieldMask

Setup GCP Parameters

In [2]:
# Locations
REGION = 'us-central1'
PROJECT_ID='statmike-mlops'
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

Clients for Feature Store:

In [3]:
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)
client_options = {"api_endpoint": API_ENDPOINT}
clients = {}

In [4]:
clients['fs'] = FeaturestoreServiceClient(client_options=client_options)

In [5]:
clients['fs_olserve'] = FeaturestoreOnlineServingServiceClient(client_options=client_options)

In [6]:
BASE_RESOURCE_PATH = clients['fs'].common_location_path(PROJECT_ID, REGION)

---
## Feature Store Data model
Feature Store organizes data with the following 3 important hierarchical concepts:

Featurestore -> EntityType -> Feature

- **Featurestore**: the place to store your features
    - **EntityType**: under a Featurestore, an EntityType describes an object to be modeled, real one or virtual one.
        - **Feature**: under an EntityType, a feature describes an attribute of the EntityType

For the digits data used in these examples, the feature store will be called digits_featurestore.  The store has 1 entity type: images.  The features will be the pixels and the target values.

---
## Create Feature Store

In [7]:
FEATURESTORE_ID = 'digits_featurestore'

In [8]:
featurestore_lro = clients['fs'].create_featurestore(
    types.featurestore_service.CreateFeaturestoreRequest(
        parent = BASE_RESOURCE_PATH,
        featurestore_id = FEATURESTORE_ID,
        featurestore=types.featurestore.Featurestore(
            display_name="Featurestore for handwritten digits",
            online_serving_config=types.featurestore.Featurestore.OnlineServingConfig(
                fixed_node_count=3
            ),
        ),
    )
)

In [9]:
featurestore_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"

Use `get_featurestore` to see details of specified feature store:

In [10]:
clients['fs'].get_featurestore(name=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID))

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"
create_time {
  seconds: 1626261161
  nanos: 384226000
}
update_time {
  seconds: 1626261161
  nanos: 675344000
}
etag: "AMEw9yMcennAU95FzDcoMtAM9HFJcWoPlxRF41-aGdVwX70AH3sJnQJDJ9ljtLtRlmQI"
online_serving_config {
  fixed_node_count: 3
}
state: STABLE

Use `list_featurestores` to see details of all feature stores:

In [11]:
clients['fs'].list_featurestores(parent=PARENT)

ListFeaturestoresPager<featurestores {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore"
  create_time {
    seconds: 1626261161
    nanos: 384226000
  }
  update_time {
    seconds: 1626261161
    nanos: 675344000
  }
  etag: "AMEw9yPR6k4wBWpBajS-1MozM142XqldGykjUFraWo9JoX88khSEjDq5xgyNYc3ZTZkt"
  online_serving_config {
    fixed_node_count: 3
  }
  state: STABLE
}
>

---
## Create Entity Type

In [12]:
ENTITYTYPE_ID = 'image'

In [13]:
entitytype_lro = clients['fs'].create_entity_type(
    types.featurestore_service.CreateEntityTypeRequest(
        parent=clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID),
        entity_type_id = ENTITYTYPE_ID,
        entity_type=types.entity_type.EntityType(
            description="image entity for digits",
            monitoring_config=types.featurestore_monitoring.FeaturestoreMonitoringConfig(
                snapshot_analysis=types.featurestore_monitoring.FeaturestoreMonitoringConfig.SnapshotAnalysis(
                    monitoring_interval=Duration(seconds=1800),  # 30 minutes
                ),
            ),
        ),
    )
)

In [14]:
entitytype_lro.result()

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image"
etag: "AMEw9yPU23Nc5-l4J_vHruRqGI2rq09sNhiuGBBGNqIo5ywovaNw"

Use `list_entity_types` to see details of all entity types:

In [15]:
clients['fs'].list_entity_types(parent=PARENT+'/featurestores/{}'.format(FEATURESTORE_ID))

ListEntityTypesPager<entity_types {
  name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image"
  description: "image entity for digits"
  create_time {
    seconds: 1626261178
    nanos: 736002000
  }
  update_time {
    seconds: 1626261178
    nanos: 736002000
  }
  etag: "AMEw9yNOjcHXYO28GrbHqEl-BDUJjrFnycgJb-K9ran167gDvnuNHeEwy1t_vsDEa6f1"
  monitoring_config {
    snapshot_analysis {
      monitoring_interval {
        seconds: 86400
      }
    }
  }
}
>

---
## Create Features

Get the schema of the data source for new features:

In [16]:
%%bigquery schema
SELECT * 
FROM `statmike-mlops.digits.INFORMATION_SCHEMA.COLUMN_FIELD_PATHS`
WHERE table_name = 'digits_source'

Downloading: 100%|██████████| 66/66 [00:01<00:00, 60.80rows/s]


In [17]:
schema

,table_catalog,table_schema,table_name,column_name,field_path,data_type,description
0,statmike-mlops,digits,digits_source,p0,p0,FLOAT64,None
1,statmike-mlops,digits,digits_source,p1,p1,FLOAT64,None
2,statmike-mlops,digits,digits_source,p2,p2,FLOAT64,None
3,statmike-mlops,digits,digits_source,p3,p3,FLOAT64,None
4,statmike-mlops,digits,digits_source,p4,p4,FLOAT64,None
...,...,...,...,...,...,...,...
61,statmike-mlops,digits,digits_source,p61,p61,FLOAT64,None
62,statmike-mlops,digits,digits_source,p62,p62,FLOAT64,None
63,statmike-mlops,digits,digits_source,p63,p63,FLOAT64,None
64,statmike-mlops,digits,digits_source,target,target,INT64,None


Prepare a request for `batch_create_features`:
- specification for the features, data type and descriptions ....

In [18]:
REQUESTS = []
for i in range(schema.shape[0]):
    
    if schema['data_type'][i] == 'STRING': value_type = types.feature.Feature.ValueType.STRING
    elif schema['data_type'][i] == 'INT64': value_type = types.feature.Feature.ValueType.INT64
    elif schema['data_type'][i] == 'FLOAT64': value_type = types.feature.Feature.ValueType.DOUBLE
    
    if schema['description'][i] == None: description = schema['column_name'][i]
    else: description = schema['description'][i]
    
    REQUESTS.append(
        types.featurestore_service.CreateFeatureRequest(
            feature=types.feature.Feature(
                value_type = value_type,
                description = description,
                # optional, monitoring_config here as override, otherwise it inherits from entity_type
            ),
            feature_id = schema['column_name'][i].lower(),
        )    
    )

In [19]:
batchfeatures = clients['fs'].batch_create_features(
    parent = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    requests = REQUESTS,
)

In [20]:
list(item.name for item in batchfeatures.result().features)

['projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p0',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p1',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p2',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p3',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p4',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p5',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p6',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p7',
 'projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/featur

---
## Search Features
Search goes across all Feature Stores and Entity Types.

Also, use the list_features function to list all.

In [21]:
# return the first feature:
list(clients['fs'].search_features(location=BASE_RESOURCE_PATH))[0]

name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p0"
description: "p0"
create_time {
  seconds: 1626261213
  nanos: 459427000
}
update_time {
  seconds: 1626261213
  nanos: 459427000
}

In [22]:
# find all features with INT64 value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="value_type=INT64")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/target"
 description: "target"
 create_time {
   seconds: 1626261213
   nanos: 656623000
 }
 update_time {
   seconds: 1626261213
   nanos: 656623000
 }]

In [23]:
# find all features of the form p6* with DOUBLE value type
list(clients['fs'].search_features(types.featurestore_service.SearchFeaturesRequest(location=BASE_RESOURCE_PATH, query="feature_id:p6* AND value_type=DOUBLE")))

[name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p6"
 description: "p6"
 create_time {
   seconds: 1626261213
   nanos: 480926000
 }
 update_time {
   seconds: 1626261213
   nanos: 480926000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p60"
 description: "p60"
 create_time {
   seconds: 1626261213
   nanos: 643930000
 }
 update_time {
   seconds: 1626261213
   nanos: 643930000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p61"
 description: "p61"
 create_time {
   seconds: 1626261213
   nanos: 647801000
 }
 update_time {
   seconds: 1626261213
   nanos: 647801000
 },
 name: "projects/691911073727/locations/us-central1/featurestores/digits_featurestore/entityTypes/image/features/p62"
 description: "p62"
 create_time {
   seconds: 1626261213
   nanos: 650150000
 }
 update_time {
   se

---
## Import Feature Values
- BigQuery (THIS DEMO)
- Avro
- CSV

Prepare a source table with timestamp (update_time) and unique id's (image_id) for each entity

In [24]:
%%bigquery
CREATE OR REPLACE TABLE `statmike-mlops.digits.digits_featurestore_import` AS
SELECT GENERATE_UUID() image_id, target_OE as target_oe, CURRENT_TIMESTAMP AS update_time, * EXCEPT(target_OE)
FROM `statmike-mlops.digits.digits_source`

Query complete after 0.00s: 100%|██████████| 3/3 [00:00<00:00, 1357.53query/s]                        


""


Create Feature specification for each feature in the input source:

In [25]:
FEATURE_SPECS = []
for i in range(schema.shape[0]):
    FEATURE_SPECS.append(types.featurestore_service.ImportFeatureValuesRequest.FeatureSpec(id=schema['column_name'][i].lower()))

In [26]:
import_request = types.featurestore_service.ImportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    bigquery_source = types.BigQuerySource(input_uri='bq://statmike-mlops.digits.digits_featurestore_import'),
    feature_time_field = "update_time",
    feature_time = Timestamp().GetCurrentTime(),
    entity_id_field = "image_id",
    feature_specs = FEATURE_SPECS,
    worker_count = 4,
)

In [27]:
importjob = clients['fs'].import_feature_values(import_request)

In [28]:
importjob.result()

imported_entity_count: 1797
imported_feature_value_count: 118602

---
## Serving Features

Retrieve a list of entity id's:

In [29]:
%%bigquery image_id
SELECT image_id FROM `statmike-mlops.digits.digits_featurestore_import`

Downloading: 100%|██████████| 1797/1797 [00:01<00:00, 1623.66rows/s]


In [31]:
id_values = list(image_id['image_id'])
id_values[0:5]

['45d42286-0679-489b-b99b-379d9978d861',
 'a0529b59-5cd3-43e3-aeb2-bfdbf60d4931',
 '4779414c-1a5e-4c5d-9b4b-328c0f818021',
 '52513cf0-a087-4304-bc16-b92db65b9cf1',
 '1d20b1b1-5d0c-4119-9007-d4d531c71d07']

### Online - One

In [32]:
single = clients['fs_olserve'].read_feature_values(
    types.featurestore_online_service.ReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_id=id_values[0],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [33]:
print(list(item.id for item in single.header.feature_descriptors))

['p22', 'p14', 'p13', 'p9', 'p49', 'p10', 'p38', 'p59', 'p11', 'p62', 'p20', 'p61', 'p56', 'p4', 'p50', 'p23', 'p0', 'p33', 'p26', 'p29', 'p37', 'p35', 'p45', 'p19', 'p5', 'p17', 'p36', 'p46', 'p58', 'p63', 'p34', 'p1', 'p15', 'p55', 'p12', 'p8', 'p30', 'p24', 'p40', 'p43', 'p2', 'p57', 'target', 'p25', 'p3', 'p31', 'p52', 'p47', 'p18', 'p7', 'p27', 'p16', 'p39', 'p42', 'p51', 'p6', 'p41', 'target_oe', 'p53', 'p21', 'p48', 'p44', 'p32', 'p54', 'p28', 'p60']


In [34]:
print(list(item.value.double_value for item in single.entity_view.data))

[6.0, 0.0, 13.0, 0.0, 0.0, 15.0, 8.0, 10.0, 12.0, 0.0, 1.0, 3.0, 0.0, 14.0, 8.0, 0.0, 0.0, 6.0, 12.0, 8.0, 5.0, 0.0, 12.0, 4.0, 4.0, 4.0, 0.0, 8.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 11.0, 0.0, 8.0, 0.0, 0.0, 1.0, 2.0, 0.0, 0.0, 4.0, 12.0, 0.0, 9.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0, 12.0, 12.0, 0.0, 3.0, 0.0, 16.0, 14.0, 0.0, 0.0, 0.0, 3.0, 0.0, 13.0]


#### Prediction

In [35]:
from google.cloud.aiplatform_v1beta1 import (PredictionServiceClient, EndpointServiceClient)
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value

Function to retrieve online features for an entity and prepare them for prediction:

In [36]:
def fetch_single(entity_id):
    single = clients['fs_olserve'].read_feature_values(
        types.featurestore_online_service.ReadFeatureValuesRequest(
            entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
            entity_id=entity_id,
            feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
        )
    )
    
    newob = {}
    features = list(item.id for item in single.header.feature_descriptors)
    for e, f in enumerate(features):
        if f.startswith('p'):
            newob[f] = single.entity_view.data[e].value.double_value
    
    return newob

In [38]:
newob = fetch_single(id_values[0])

Pick an endpoint from those setup in this project (all have the same model parameters):

In [39]:
clients['endpoint'] = EndpointServiceClient(client_options=client_options)

In [40]:
list((ep.name, ep.display_name) for ep in clients['endpoint'].list_endpoints(parent=PARENT))

[('projects/691911073727/locations/us-central1/endpoints/4598931683545186304',
  'bq_digits_code_20210714100524'),
 ('projects/691911073727/locations/us-central1/endpoints/6904774692758880256',
  'ENDPOINT_04_KERAS-DIGITS'),
 ('projects/691911073727/locations/us-central1/endpoints/1336073753515261952',
  'ENDPOINT_03_BQML-DIGITS')]

In [41]:
ep = clients['endpoint'].list_endpoints(parent=PARENT).endpoints[1].name
ep

'projects/691911073727/locations/us-central1/endpoints/6904774692758880256'

Python: Request a prediction for the entity's features:

In [42]:
clients['prediction'] = PredictionServiceClient(client_options=client_options)

In [43]:
response = clients['prediction'].predict(endpoint=ep, instances=[json_format.ParseDict(newob, Value())], parameters=json_format.ParseDict({}, Value()))

In [44]:
response

predictions {
  list_value {
    values {
      number_value: 0.999998093
    }
    values {
      number_value: 3.23975326e-11
    }
    values {
      number_value: 9.81113146e-08
    }
    values {
      number_value: 8.48128456e-10
    }
    values {
      number_value: 5.70161269e-08
    }
    values {
      number_value: 5.66611646e-09
    }
    values {
      number_value: 1.72426839e-09
    }
    values {
      number_value: 1.39610012e-08
    }
    values {
      number_value: 1.62476601e-06
    }
    values {
      number_value: 1.57111018e-07
    }
  }
}
deployed_model_id: "8112091236615061504"

REST: Request a prediction for the entity's features:

In [45]:
import json
with open('request.json','w') as file:
    file.write(json.dumps({"instances": [newob]}))

In [47]:
!curl -X POST \
-H "Authorization: Bearer "$(gcloud auth application-default print-access-token) \
-H "Content-Type: application/json; charset=utf-8" \
-d @request.json \
https://{API_ENDPOINT}/v1/{ep}:predict

{
  "predictions": [
    [
      0.999998093,
      3.23975326e-11,
      9.81113146e-08,
      8.48128456e-10,
      5.70161269e-08,
      5.66611646e-09,
      1.72426839e-09,
      1.39610012e-08,
      1.62476601e-06,
      1.57111018e-07
    ]
  ],
  "deployedModelId": "8112091236615061504"
}


CLI: Request a prediction for the entity's features:

In [48]:
!gcloud beta ai endpoints predict {ep.rsplit('/',1)[-1]} --region={REGION} --json-request=request.json

Using endpoint [https://us-central1-prediction-aiplatform.googleapis.com/]
[[0.999998093, 3.23975326e-11, 9.81113146e-08, 8.48128456e-10, 5.70161269e-08, 5.66611646e-09, 1.72426839e-09, 1.39610012e-08, 1.62476601e-06, 1.57111018e-07]]


### Online - Multi

In [49]:
multi = clients['fs_olserve'].streaming_read_feature_values(
    types.featurestore_online_service.StreamingReadFeatureValuesRequest(
        entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        entity_ids = id_values[0:3],
        feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*'])),
    )
)

In [50]:
for i in multi:
    print(i.entity_view.entity_id)
    print(list(item.value.double_value for item in i.entity_view.data))


[]
45d42286-0679-489b-b99b-379d9978d861
[16.0, 0.0, 12.0, 0.0, 0.0, 8.0, 6.0, 12.0, 12.0, 15.0, 0.0, 8.0, 0.0, 0.0, 3.0, 1.0, 0.0, 11.0, 14.0, 1.0, 0.0, 4.0, 12.0, 3.0, 0.0, 0.0, 12.0, 0.0, 14.0, 13.0, 2.0, 0.0, 0.0, 4.0, 0.0, 6.0, 8.0, 0.0, 0.0, 0.0, 3.0, 0.0, 8.0, 16.0, 0.0, 12.0, 0.0, 0.0, 13.0, 0.0, 0.0, 5.0, 8.0, 9.0, 0.0, 0.0, 10.0, 4.0, 0.0, 0.0, 0.0, 9.0, 0.0, 0.0, 0.0, 4.0]
4779414c-1a5e-4c5d-9b4b-328c0f818021
[14.0, 0.0, 16.0, 0.0, 0.0, 9.0, 0.0, 16.0, 15.0, 6.0, 0.0, 3.0, 0.0, 0.0, 8.0, 13.0, 0.0, 14.0, 11.0, 1.0, 0.0, 15.0, 15.0, 0.0, 0.0, 0.0, 16.0, 10.0, 12.0, 16.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 15.0, 14.0, 15.0, 0.0, 16.0, 0.0, 0.0, 7.0, 0.0, 3.0, 14.0, 2.0, 16.0, 0.0, 0.0, 13.0, 0.0, 0.0, 0.0, 0.0, 16.0, 0.0, 0.0, 0.0, 0.0]
a0529b59-5cd3-43e3-aeb2-bfdbf60d4931
[14.0, 0.0, 8.0, 0.0, 0.0, 7.0, 3.0, 16.0, 13.0, 15.0, 0.0, 2.0, 0.0, 0.0, 4.0, 5.0, 0.0, 13.0, 12.0, 1.0, 0.0, 3.0, 15.0, 1.0, 0.0, 0.0, 12.0, 0.0, 15.0, 14.0, 2.0, 0.0, 0.0, 3.0, 0.0, 7

### Batch (For training or large scale prediction)

In [51]:
from google.cloud import bigquery

DESTINATION_DATASET = 'digits_training'

clients['bq'] = bigquery.Client()
dataset_id = "{}.{}".format(clients['bq'].project, DESTINATION_DATASET)
dataset = bigquery.Dataset(dataset_id)
dataset.location = REGION
dataset = clients['bq'].create_dataset(dataset, exists_ok = True)

In [52]:
batch_request = types.featurestore_service.ExportFeatureValuesRequest(
    entity_type = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
    snapshot_export = types.ExportFeatureValuesRequest.SnapshotExport(snapshot_time = Timestamp().GetCurrentTime()),
    destination = types.FeatureValueDestination(bigquery_destination = types.BigQueryDestination(output_uri='bq://statmike-mlops.digits_training.training')),
    feature_selector = types.FeatureSelector(id_matcher=types.IdMatcher(ids=['*']))
)

In [53]:
batchjob = clients['fs'].export_feature_values(batch_request)

In [55]:
batchjob.result()

---
## Clean Up

In [256]:
clients['fs'].delete_feature(
    name = PARENT + '/featurestores/{}/entityTypes/{}/features/p0'.format(FEATURESTORE_ID,ENTITYTYPE_ID)
).result()

In [257]:
clients['fs'].delete_feature(
    name = clients['fs'].feature_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID, 'p1')
).result()

In [282]:
clients['fs'].delete_entity_type(
    request = types.DeleteEntityTypeRequest(
        name = clients['fs'].entity_type_path(PROJECT_ID, REGION, FEATURESTORE_ID, ENTITYTYPE_ID),
        force = True)
).result()

In [283]:
clients['fs'].delete_featurestore( name = clients['fs'].featurestore_path(PROJECT_ID, REGION, FEATURESTORE_ID))

In [262]:
clients['bq'].delete_dataset(dataset, delete_contents = True)